### 1 - Import test data

In [58]:
import yfinance as yf
import pandas as pd

dataF = yf.download("EURUSD=X", start="2025-09-7", end="2025-11-2", interval='15m')
dataF.iloc[:,:]
#dataF.Open.iloc

/var/folders/pp/x__1tf4n7j98nw_8bwlmgxj00000gn/T/ipykernel_4948/1723075212.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  dataF = yf.download("EURUSD=X", start="2025-09-7", end="2025-11-2", interval='15m')
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,EURUSD=X,EURUSD=X,EURUSD=X,EURUSD=X,EURUSD=X
Datetime,,,,,
2025-09-07 23:00:00+00:00,1.171097,1.171372,1.171097,1.171097,0
2025-09-07 23:15:00+00:00,1.171372,1.171372,1.171097,1.171097,0
2025-09-07 23:30:00+00:00,1.171646,1.171646,1.171372,1.171372,0
2025-09-07 23:45:00+00:00,1.171646,1.171921,1.171509,1.171921,0
2025-09-08 00:00:00+00:00,1.170960,1.171234,1.170960,1.171097,0
...,...,...,...,...,...
2025-10-31 20:15:00+00:00,1.153536,1.153669,1.153136,1.153270,0
2025-10-31 20:30:00+00:00,1.153536,1.153669,1.153403,1.153669,0


In [ ]:
my_data = dataF[['Open', 'High', 'Low', 'Close']]
my_data

In [ ]:
def flatten_yf_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Removes the 'Ticker' row level from a Yahoo Finance DataFrame
    that has multi-index columns like ('EURUSD=X', 'Open').

    Returns a DataFrame with simple column names like 'Open', 'High', etc.
    """
    # If columns are MultiIndex (e.g. ('EURUSD=X', 'Open')), flatten them
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(0)
    return df


my_data  = flatten_yf_columns(my_data)
my_data.columns = ['Open', 'High', 'Low', 'Close']
my_data

### 2 - Define your signal function

In [ ]:
# def signal_generator(df):
#     open = df.Open.iloc[-1]
#     close = df.Close.iloc[-1]
#     previous_open = df.Open.iloc[-2]
#     previous_close = df.Close.iloc[-2]
    
#     # Bearish Pattern
#     if (open>close and 
#     previous_open<previous_close and 
#     close<previous_open and
#     open>=previous_close):
#         return 1

#     # Bullish Pattern
#     elif (open<close and 
#         previous_open>previous_close and 
#         close>previous_open and
#         open<=previous_close):
#         return 2
    
#     # No clear pattern
#     else:
#         return 0

# signal = []
# signal.append(0)
# for i in range(1, len(my_data)):
#     df = dataF[i-1:i+1]
#     signal.append(signal_generator(df))
# #signal_generator(data)
# dataF["signal"] = signal

import pandas as pd


def signal_generator(df):
    # Safety: only process if at least 2 rows
    if len(df) < 2:
        return 0

    # Extract the last and previous rows explicitly and force scalar values
    last = df.iloc[-1]
    prev = df.iloc[-2]

    open_ = float(last["Open"])
    close_ = float(last["Close"])
    previous_open = float(prev["Open"])
    previous_close = float(prev["Close"])

    # Bearish Pattern
    if (open_ > close_ and
        previous_open < previous_close and
        close_ < previous_open and
            open_ >= previous_close):
        return 1

    # Bullish Pattern
    elif (open_ < close_ and
          previous_open > previous_close and
          close_ > previous_open and
          open_ <= previous_close):
        return 2

    # No clear pattern
    else:
        return 0


signal = [0]
for i in range(1, len(dataF)):
    df_slice = dataF.iloc[i-1:i+1]
    signal.append(signal_generator(df_slice))

dataF["signal"] = signal

In [ ]:
dataF.signal.value_counts()
#dataF.iloc[:, :]

### 3 - Connect to the market and execute trades

In [ ]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

In [ ]:
from config import access_token, accountID
def get_candles(n):
    #access_token='XXXXXXX'#you need token here generated from OANDA account
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.EUR_USD, Gran.M15)
    candles = collector.grab(n)
    return candles

candles = get_candles(3)
for candle in candles:
    print(float(str(candle.bid.o))>1)


In [ ]:
def trading_job():
    candles = get_candles(3)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low'])
    
    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i=i+1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    signal = signal_generator(dfstream.iloc[:-1,:])#
    
    # EXECUTING ORDERS
    #accountID = "XXXXXXX" #your account ID here
    client = API(access_token)
         
    SLTPRatio = 2.
    previous_candleR = abs(dfstream['High'].iloc[-2]-dfstream['Low'].iloc[-2])
    
    SLBuy = float(str(candle.bid.o))-previous_candleR
    SLSell = float(str(candle.bid.o))+previous_candleR

    TPBuy = float(str(candle.bid.o))+previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o))-previous_candleR*SLTPRatio
    
    print(dfstream.iloc[:-1,:])
    print(TPBuy, "  ", SLBuy, "  ", TPSell, "  ", SLSell)
    # signal = 2
    #Sell
    if signal == 1:
        mo = MarketOrderRequest(instrument="EUR_USD", units=-1050, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)
    #Buy
    elif signal == 2:
        mo = MarketOrderRequest(instrument="EUR_USD", units=1050, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

### 4 - Executing orders automatically with a scheduler

In [57]:
import datetime
import pytz
# trading_job()

scheduler = BlockingScheduler()
scheduler.add_job(trading_job, 'cron', day_of_week='mon-fri', hour='00-23', minute='1,16,31,46',
                  start_date=datetime.datetime.now(pytz.timezone('America/Chicago')), timezone='America/Chicago')
scheduler.start()

      Open    Close     High      Low
0  1.15161  1.15125  1.15168  1.15088
1  1.15127  1.15118  1.15162  1.15103
1.1523600000000003    1.15059    1.15    1.1517700000000002
      Open    Close     High      Low
0  1.15127  1.15118  1.15162  1.15103
1  1.15118  1.15178  1.15180  1.15113
1.1531099999999999    1.1511    1.15043    1.15244
{'orderCreateTransaction': {'id': '17', 'accountID': '101-001-37532546-001', 'userID': 37532546, 'batchID': '17', 'requestID': '79448638259973594', 'time': '2025-11-03T09:31:02.275707049Z', 'type': 'MARKET_ORDER', 'instrument': 'EUR_USD', 'units': '1050', 'timeInForce': 'FOK', 'positionFill': 'DEFAULT', 'takeProfitOnFill': {'price': '1.15311', 'timeInForce': 'GTC'}, 'stopLossOnFill': {'price': '1.15110', 'timeInForce': 'GTC', 'triggerMode': 'TOP_OF_BOOK'}, 'reason': 'CLIENT_ORDER'}, 'orderFillTransaction': {'id': '18', 'accountID': '101-001-37532546-001', 'userID': 37532546, 'batchID': '17', 'requestID': '79448638259973594', 'time': '2025-11-03T09:31:02

KeyboardInterrupt: 